In [1]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score
from multiprocessing import Pool
import scipy as sc
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns =100
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import gensim
import pickle
from gensim.models import Word2Vec, FastText, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import sys
sys.path.insert(0, '..')
from Common import preprocessing,evaluation,CosineClassifier as cos
classes_map = {'DOC':0, 'ENTER':1, 'ORG':2, 'PRIV':3, 'RANG':4, 'HOST':5}

In [66]:
df = pd.read_csv('..//Data//data.txt', delimiter=';', engine='python',encoding='utf8')

In [67]:
questions = np.array(df.question)
questions = preprocessing.preprocess_list(questions)

In [135]:
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,1))
X = vectorizer.fit_transform(questions)

In [125]:
classes = np.array(df['class'])
y = list(map(lambda x: classes_map[x],classes))

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,shuffle=True)

In [56]:
log_reg = OneVsRestClassifier(LogisticRegression(random_state=0,C=10,solver='lbfgs',)).fit(X_train, y_train)
ridge = OneVsRestClassifier(RidgeClassifier(random_state=0)).fit(X_train, y_train)
svc = OneVsRestClassifier(LinearSVC(random_state=0,)).fit(X_train, y_train)
#clf = cos.CosineClassifier().fit(X_train,y_train)

In [57]:
print(evaluation.get_CV_scores(log_reg,X_test,y_test,'precision_micro').mean())
print(evaluation.get_CV_scores(ridge,X_test,y_test,'precision_micro').mean())
print(evaluation.get_CV_scores(svc,X_test,y_test,'precision_micro').mean())

0.9674157303370787
0.9662921348314606
0.9685393258426966


In [7]:
count_model = CountVectorizer(min_df=3,ngram_range=(1,1))
X_ = count_model.fit_transform(questions)
Xc = (X_.T * X_)
Xc.setdiag(0)
print(Xc.todense().shape)

(2076, 2076)


In [20]:
model = Word2Vec([q.split() for q in questions], size=300, window=10,)
model.train([q.split() for q in questions],epochs=200,total_examples=model.corpus_count)
#model = KeyedVectors.load_word2vec_format("..//..//web_upos_cbow_300_20_2017.bin.gz", binary=True, unicode_errors='ignore')

(4905495, 10625600)

In [21]:
word2vec = []

for q in questions:
    vec = [model.wv[w] for w in q.split() if w in model.wv]
    if len(vec) < 1:
        word2vec.append(np.zeros(300))
    else:
        word2vec.append(np.array(np.mean(vec,axis=0)))

In [38]:
embeddings = []

for q in questions:
    vec = [enth[w][0] for w in q.split() if w in list(enth.keys())]
    if len(vec) < 1:
        embeddings.append(np.zeros(300))
    else:
        embeddings.append(np.array(np.mean(vec,axis=0)))

In [9]:
co_occur = Normalizer().fit_transform(Xc.todense())

In [10]:
enthropy = []
for q in questions:
    question = q.split()
    q_dic = Counter(question)
    q_len = len(question)
    vector = []
    cnt=0
    for w in vectorizer.get_feature_names():
            if w in question:
                vector.append(sum(co_occur[cnt])*q_dic[w]/q_len*np.log2(q_dic[w]/q_len))
            else:
                vector.append(-0.00001*sum(co_occur[cnt]))
            cnt=cnt+1
    enthropy.append(vector.copy())

In [35]:
svd = TruncatedSVD(n_components=400,algorithm='arpack')
matr = svd.fit_transform(np.array(enthropy).T)

In [36]:
matr = Normalizer().fit_transform(matr)

In [37]:
enth = {}
for i in list(zip(vectorizer.get_feature_names(),matr)):
    enth[i[0]]=[i[1]]

In [134]:
pd.DataFrame(df).to_csv("labels_.tsv", sep='\t',index=False)

In [2]:
df = pd.read_csv('..//Data//ccg.txt',sep=':', delimiter=':', engine='python',encoding='utf8',error_bad_lines=False)

Skipping line 259: Expected 2 fields in line 259, saw 3
Skipping line 288: Expected 2 fields in line 288, saw 3
Skipping line 299: Expected 2 fields in line 299, saw 4
Skipping line 372: Expected 2 fields in line 372, saw 3
Skipping line 374: Expected 2 fields in line 374, saw 3
Skipping line 419: Expected 2 fields in line 419, saw 3
Skipping line 477: Expected 2 fields in line 477, saw 3
Skipping line 527: Expected 2 fields in line 527, saw 3
Skipping line 591: Expected 2 fields in line 591, saw 3
Skipping line 595: Expected 2 fields in line 595, saw 3
Skipping line 696: Expected 2 fields in line 696, saw 3
Skipping line 794: Expected 2 fields in line 794, saw 3
Skipping line 862: Expected 2 fields in line 862, saw 3
Skipping line 878: Expected 2 fields in line 878, saw 3
Skipping line 931: Expected 2 fields in line 931, saw 3
Skipping line 1004: Expected 2 fields in line 1004, saw 3
Skipping line 1114: Expected 2 fields in line 1114, saw 3
Skipping line 1307: Expected 2 fields in lin

In [5]:
questions = np.array(df.question)
questions = preprocessing.preprocess_eng_list(questions)
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,1))
X = vectorizer.fit_transform(questions)
classes = np.array(df['class'])
y = list(map(lambda x: classes_map2[x],classes))

In [4]:
classes_map2 = {'ENTY':0,'HUM':1,'DESC':2,'NUM':3,'LOC':4,'ABBR':5}

In [45]:
X.shape

(5392, 2076)